In [ ]:
# coding: utf-8
import ast
import json
import os
import sys
import re
from pprint import pprint
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import datetime


from func import blast 

run_name = 'test'

PATH = os.path.abspath(".") # Path to Where you want to have the data. Default is in the BFG-PCA folder. 

#################################
## Preparing Directories for Run
if not os.path.isdir('%s/Data'%(PATH)):
    os.makedirs('%s/Data'%(PATH))
if os.path.isdir('%s/Data/%s'%(PATH,run_name)):
    print ('%s/Data/%s already exists.\nPlease delete the previous data, or change the run_name.'%(PATH,run_name))
if not os.path.isdir('%s/Data/%s'%(PATH,run_name)):
    os.makedirs('%s/Data/%s'%(PATH,run_name))
    os.makedirs('%s/Data/%s/fragmented_fasta'%(PATH,run_name))
    os.makedirs('%s/Data/%s/blast'%(PATH,run_name))
    os.makedirs('%s/Data/%s/blast/sh'%(PATH,run_name))
    os.makedirs('%s/Data/%s/blast/out'%(PATH,run_name))
    os.makedirs('%s/Data/%s/barcode_counts'%(PATH,run_name))
    os.makedirs('%s/Data/%s/db'%(PATH,run_name))

In [ ]:
###############################
## Create BLAST database
databse   = "%s/test/test_database.csv"%(PATH) #Change this to your own database
const_seq = "%s/const-seq.csv"%(PATH) # DO NOT CHANGE THIS

db_fna    = blast.mkdb(databse,const_seq,run_name,"%s/Data/%s/db"%(PATH,run_name))

command = "makeblastdb -in %s  -dbtype nucl"%(db_fna)

print(os.popen(command).read())

In [ ]:
################################
## Convert fastq to fasta files in small chunks to make it easy to handle
miseq_dir =  #'FULL/PATH/TO/FASTQ' Put both R1 and R2 fastqs in the directory. Do not forget to uncompress the files if they were compressed. 
blast.fastq2fasta(miseq_dir,150,"%s/Data/%s"%(PATH,run_name))

In [ ]:
#################################
## Create sh files to execute BLAST
blast.mk_blast_sh('%s/Data/%s/fragmented_fasta'%(PATH,run_name),db_fna, '%s/Data/%s'%(PATH,run_name))


In [ ]:
################################
## Execute BLAST
print("Executing BLAST for barcode alignment...(this takes time)\n")
print("Start time: %s"%datetime.datetime.now())
blast_sh = blast.get_sh('%s/Data/%s/blast/sh'%(PATH,run_name))

for sh in tqdm(blast_sh):
    command = "sh %s/Data/%s/blast/sh/%s"%(PATH,run_name,sh)
    os.system(command)
    os.system("rm %s/Data/%s/blast/sh/%s"%(PATH,run_name,sh))
#This script executes the sh files one-by-one.
#If you have multiple threads available, you can execute the sh files by yourself. 

In [ ]:
################################
## Parse BLAST output

print("Parsing BLAST alignment results...")

blast.parse_blast_out(PATH,run_name,db_fna)

        
print("\n\n\nProceed to BFG_data_normalization.ipynb to perform score normalization.")